In [2]:
# import libraries
#these two lines only in colab
from google.colab import drive
drive.mount('/content/gdrive')
#-------------------------------------------------
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import zipfile
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Conv1D, MaxPooling1D

from sklearn.model_selection import train_test_split

Mounted at /content/gdrive


In [ ]:
pfad = "C:/Users/sebas/Downloads/1daee22f3f13fe6bc6a343f829565759-3511dc6de6a7bf064c168b4f20b85a20d8f83b91.zip"

In [ ]:
with zipfile.ZipFile(pfad, 'r') as zip_ref:
    zip_ref.extractall('C:/Users/sebas/Downloads/')

In [ ]:
folder = 'C:/Users/sebas/Downloads/1daee22f3f13fe6bc6a343f829565759-3511dc6de6a7bf064c168b4f20b85a20d8f83b91/'

In [ ]:
folder = '/content/gdrive/My Drive/textos/'

In [3]:
folder = '/content/gdrive/My Drive/corpus/'

In [ ]:
folder = ('C:\\Users\\sebas\\Google Drive\\corpus\\').replace('\\','/') #this only in local device

In [4]:
filenames = os.listdir(folder)
print(filenames)

text_file = 'cien_annos_soledad.txt'

['PRUEBAW2V.txt', 'cien-años-de-soledad.txt', 'articulo-wikipedia.txt', 'cat-and-dogs.txt', 'el-principito.txt', 'wikipedia_perro.txt', 'la-isla-del-tesoro.txt', 'jesica-cardiologia.txt', 'el-quijote.txt', 'los-tres-mosqueteros.txt', 'cien_annos_soledad.txt', 'Texto.txt']


In [ ]:
with open(folder+'Texto.txt', 'w') as outfile:
    for fname in filenames:
        if fname != 'Texto.txt':
            with open(folder+fname, encoding='utf-8') as infile:
                for line in infile:
                    outfile.write(line)

In [5]:
#text = open(folder+'la_biblioteca_de_babel.txt', encoding = 'utf-8').read().lower()
text = open(folder+text_file, encoding = 'utf-8').read().lower()
text = text.split() #this makes sure to pick up the words
text = ' '.join(text) #this takes all the letters without the separators

In [6]:
corpus_length = len(text) #number of words
corpus_length

814933

In [7]:
#clean text
text = text[120:-142]

In [8]:
 #number of distinct characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print('Number of unique characters: ', vocab_size)

Number of unique characters:  60


In [9]:
maxlen = 100 #max length of characters in each sentence 
step = 1
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

Number of sequences: 814571


In [10]:
print(sentences[50])
print(next_chars[50])
print(sentences[50])

 años después, frente al pelotón de fusilamiento, el coronel aureliano buendía había de recordar aqu
e
 años después, frente al pelotón de fusilamiento, el coronel aureliano buendía había de recordar aqu


In [11]:
print(sentences[10]+next_chars[10]) #example of text completition of three characters

de soledad gabriel garcía márquez muchos años después, frente al pelotón de fusilamiento, el coronel 


In [12]:
#dictionary from characters to indices:
char_indices = {char: chars.index(char) for char in chars}

In [13]:
#array of different available characters:
char_array = np.array(chars)
char_array

array([' ', '!', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4',
       '5', '6', '7', '8', '9', ':', ';', '<', '?', 'a', 'b', 'c', 'd',
       'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q',
       'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '¡', '«', '»',
       '¿', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü'], dtype='<U1')

In [14]:
#array of encoded characters according to the dictionary:
chars_encoded = np.array([char_indices[ch] for ch in char_array], dtype='int32')
chars_encoded

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59], dtype=int32)

In [15]:
#dictionary from indices to characters:
indices_char = {idx: chars[idx] for idx in chars_encoded}

In [16]:
#encoding of the whole text:
text_encoded = np.array([char_indices[ch] for ch in text], dtype='int32')

In [17]:
#the encoded sentences represent the inputs:
sentences_encoded = []
for sentence in sentences:
    sentences_encoded.append([char_indices[ch] for ch in sentence])
    
sentences_encoded = np.array(sentences_encoded, dtype = 'int32')

In [18]:
#encoding of the complementary characters (target)
next_chars_encoded = [char_indices[ch] for ch in next_chars]

In [19]:
#this is going to be the target variable:
next_chars_encoded = np.array(next_chars_encoded, dtype = 'int32')

In [ ]:
#another approach for splitting the data:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = split_input_target(text)

In [20]:
#variables for training:
X = sentences_encoded
y = next_chars_encoded

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 1) #better not to split (optional!!)

In [22]:
#create the model
model = tf.keras.Sequential()
model.add(Embedding(input_dim = vocab_size + 2 , output_dim = 256)) #optional = maxlen+2, 256, 1024 units
model.add(keras.layers.GRU(512))
#model.add(LSTM(512))
#model.add(keras.layers.Dropout(0.5))
#model.add(keras.layers.Dense(128, activation = 'relu'))
model.add(Dense(vocab_size)) #it is better to return the logits for numerical stability.

model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         15872     
_________________________________________________________________
gru (GRU)                    (None, 512)               1182720   
_________________________________________________________________
dense (Dense)                (None, 60)                30780     
Total params: 1,229,372
Trainable params: 1,229,372
Non-trainable params: 0
_________________________________________________________________


In [23]:
history = model.fit(X_train, y_train, epochs = 10, validation_data=(X_test, y_test) ,batch_size=64)
#history = model.fit(X, y, epochs = 10, batch_size=64) #this works way better

Epoch 1/10
  31/8910 [..............................] - ETA: 3:21:55 - loss: 3.4270 - accuracy: 0.1487

KeyboardInterrupt: ignored

In [ ]:
score = model.evaluate(X,y)

25456/25456 [==============================] - 110s 4ms/step - loss: 1.2805 - accuracy: 0.6028


In [ ]:
def sample(model, starting_string, len_generated_text=100, max_input_length=maxlen, scale_factor=1.0):
    encoded_input = [char_indices[char] for char in starting_string]
    encoded_input = tf.reshape(encoded_input,(1,-1)) #convert to row vector
    
    generated_string = starting_string
    
    model.reset_states() #reset hidden states of the RNN
    
    for i in range(len_generated_text):
        logits = model(encoded_input)*scale_factor
        #logits = tf.squeeze(logits,0) 
        new_char_index = tf.random.categorical(logits=logits, num_samples = 1) #create random samples out of the logits distribution
        new_char_index = tf.squeeze(new_char_index).numpy() #eliminate redundant dimension and transform to np format
        
        generated_string += char_array[new_char_index]
        
        encoded_input = [char_indices[char] for char in generated_string ]
        encoded_input = tf.reshape(encoded_input,(1,-1))
        
    return generated_string
        

In [ ]:
tf.random.set_seed(1)
temperature = 0.3
sample(model, starting_string = 'el ', scale_factor = 1/temperature, len_generated_text=600)

In [ ]:
model.save('/content/gdrive/My Drive/text_gen_sp2.h5')

In [ ]:
os.listdir('/content')

['.config', 'gdrive', 'sample_data']

In [ ]:
data = X, y

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(data)

In [ ]:
# Batch size
BATCH_SIZE = 512

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [ ]:
dataset

<BatchDataset shapes: ((512, 100), (512,)), types: (tf.int32, tf.int32)>